In [ ]:
##########
#name of subreddit, change this
subName = "wallstreetbets"
#########################################








import pandas

##################################Cleaning Posts/Submissions file###############################
postsDF = pandas.read_csv("Sub-wallstreetbets-010121-080121-CreatedOn-230321-154649.csv")

postsDF.columns = ['author', 'id', 'created_utc', 'title', 'selftext', 'score']
print("Shape of Posts:", postsDF.shape)

####################Cleaning#######################

#make copy before cleaning
cleanedPostsDF = postsDF.copy()

#Removed deleted selftext of post meaning that the author or moderator deleted their post.
cleanedPostsDF = cleanedPostsDF.drop(cleanedPostsDF[cleanedPostsDF.selftext == '[removed]'].index)
cleanedPostsDF = cleanedPostsDF.drop(cleanedPostsDF[cleanedPostsDF.author == '[deleted]'].index)
print("After Removed:", cleanedPostsDF.shape)
###################################################

#print head
cleanedPostsDF.head()

In [ ]:
############################################Comments and cleaning comments###########################

cmmntsDF = pandas.read_csv("Comments-wallstreetbets-010121-270221-CreatedOn-240321-225631.csv")
cmmntsDF.columns = ['author', 'id', 'created_utc', 'is_submitter', 'body', 'link_id', 'parent_id', 'score', 'total_awards_received', 'banned_at_utc']
print("Shape of Comments:", cmmntsDF.shape)

cleanedCmmntsDF = cmmntsDF.copy()

####################Cleaning#######################

#Removed deleted body of comment meaning that the author or moderator deleted their post.
cleanedCmmntsDF = cleanedCmmntsDF.drop(cleanedCmmntsDF[cleanedCmmntsDF.body == '[removed]'].index)

#Remove comments where the comment is made by the author of the post. i.e remove self edges.
cleanedCmmntsDF = cleanedCmmntsDF.loc[cleanedCmmntsDF['is_submitter'] == False]
###################################################


print("Shape after clean:", cleanedCmmntsDF.shape)
cleanedCmmntsDF.head()

In [ ]:
################################################Pruning nodes################################################

#Comment Pruning
#Remove those that made only one comment
authorsFreq = cleanedCmmntsDF.copy()
authorsFreq = authorsFreq['author'].value_counts() > 1


#Convert back to dataframe
filteredAuth = authorsFreq.to_frame()
filteredAuth.reset_index(inplace=True)

#Set Column names 
filteredAuth.columns = ['author', 'tokeep']


#Get list of authors to keep
filteredAuth = filteredAuth[filteredAuth['tokeep'] == True]
listofAuth = filteredAuth['author'].tolist()

#Filter
filteredCom = cleanedCmmntsDF[cleanedCmmntsDF['author'].isin(listofAuth)]



####################Clean posts################################################

#############Random sample the posts######################
# cleanedPostsDF = cleanedPostsDF.sample(n = 100)
##########################################################

#All posts in feb
# cleanedPostsDF = cleanedPostsDF[cleanedPostsDF['created_utc'] > 1612137600]

#First Week in Jan, 1st Jan to 8th Jan 00:00 and score more than 1
print(cleanedPostsDF.shape)
cleanedPostsDF = cleanedPostsDF[(cleanedPostsDF['created_utc'] > 1609459200) & (cleanedPostsDF['created_utc'] < 1610064000)]
cleanedPostsDF = cleanedPostsDF[(cleanedPostsDF['score'] > 1)]
print(cleanedPostsDF.shape)



print("Before filter shape:",cleanedCmmntsDF.shape)
print("Filtered comments shape:",filteredCom.shape)
filteredCom.head()

In [ ]:
#Merge and get comments to posts

#Clean before merge
#Note the link_id in the comments csv from the data download starts with 't3_'
#We need to remove the first 3 chars to remove this so that we can link comments 
#with posts with a merge join.
cleanFilteredCom = filteredCom.copy()
cleanFilteredCom['link_id'] = cleanFilteredCom['link_id'].str[3:]


#Merge Join
merged = pandas.merge(cleanedPostsDF, cleanFilteredCom, left_on='id', right_on='link_id')
print("Shape of Merge:", merged.shape)

merged.head()


##Saving Files
from pathlib import Path
Path("Cleaned Posts and Comments Data/With AutoMod/Cleaned Posts").mkdir(parents=True, exist_ok=True)
Path("Cleaned Posts and Comments Data/With AutoMod/Cleaned Comments").mkdir(parents=True, exist_ok=True)

#Get Posts Data only of the cleaned posts, from the 1,000s of posts
postsDataDF = merged[['author_x', 'id_x', 'created_utc_x', 'title', 'selftext','score_x']]
postsDataDF.columns = ['author', 'id', 'created_utc', 'title', 'selftext', 'score']
postsDataDF = postsDataDF.drop_duplicates()
postsDataDF.to_csv("Cleaned Posts and Comments Data/With AutoMod/Cleaned Posts/cleanedPostsData-WITHAutoMod-{}.csv".format(subName), index=False)



#Get Comments Data only of the cleaned posts, from the 1,000,000 comments which has all the comments
commentsOfPostsDF = merged[['author_y', 'id_y', 'created_utc_y', 'is_submitter', 'body', 'link_id','parent_id', 'score_y', 'total_awards_received', 'banned_at_utc']]
commentsOfPostsDF.columns = ['author', 'id', 'created_utc', 'is_submitter', 'body', 'link_id', 'parent_id', 'score', 'total_awards_received', 'banned_at_utc']
commentsOfPostsDF.to_csv("Cleaned Posts and Comments Data/With AutoMod/Cleaned Comments/cleanedCommentsofPostsData-WITHAutoMod-{}.csv".format(subName), index=False)

In [ ]:
from pathlib import Path
Path("Edgelists/With Automod").mkdir(parents=True, exist_ok=True)

############Edge List from merged###################

edgeList = merged.copy()
edgeList = edgeList[["author_y", "author_x"]]
edgeList.columns = ["Source", "Target"]

#Drop duplicates but add a edge weight columns
#Maybe switch to edge weight?
print("With duplicate edges:", edgeList.shape)
# edgeList.drop_duplicates(subset=["Source", "Target"], keep='first',inplace=True)
#Weight Column
edgeList = edgeList.groupby(['Source', 'Target']).size().sort_values(ascending=False).reset_index(name='Weight')
print("Without duplicate edges and with edge weight column:", edgeList.shape)


edgeList.to_csv("Edgelists/With Automod/{}-EdgeList-WithAutoMod-CommentsToPosts.csv".format(subName), index=False)
edgeList.head()

In [ ]:
from pathlib import Path
Path("Cleaned Posts and Comments Data/No AutoMod/Cleaned Posts").mkdir(parents=True, exist_ok=True)
Path("Cleaned Posts and Comments Data/With AutoMod/Cleaned Comments").mkdir(parents=True, exist_ok=True)
Path("Edgelists/No AutoMod").mkdir(parents=True, exist_ok=True)
################Removed AutoMod Version of Files#########################

#No AutoMod Posts, verifiied working
print("With AutoMod Posts Shape: ", postsDataDF.shape)
noAutoModPosts = postsDataDF[postsDataDF.author != "AutoModerator"]
noAutoModPosts.to_csv("Cleaned Posts and Comments Data/No AutoMod/Cleaned Posts/PostsOnly-{}-NOAutoMod.csv".format(subName), index=False)
print("No AutoMod Posts Shape: ", noAutoModPosts.shape)


#No AutoMod Comments, need to remove comments made on an automod post so...
# need to use the 'merged' dataframe from before
print("With AutoMod Comments Shape: ", commentsOfPostsDF.shape)
noAutoModCommments = merged.copy()
noAutoModCommments = noAutoModCommments[(noAutoModCommments != 'AutoModerator').all(1)]
noAutoModCommments = noAutoModCommments[['author_y', 'id_y', 'created_utc_y', 'is_submitter', 'body', 'link_id','parent_id', 'score_y', 'total_awards_received', 'banned_at_utc']]
noAutoModCommments.columns = ['author', 'id', 'created_utc', 'is_submitter', 'body', 'link_id', 'parent_id', 'score', 'total_awards_received', 'banned_at_utc']
noAutoModCommments.to_csv("Cleaned Posts and Comments Data/No AutoMod/Cleaned Comments/CommentsOnly-{}-NOAutoMod.csv".format(subName), index=False)
print("No AutoMod Comments Shape: ", noAutoModCommments.shape)

#No AutoMod EdgeList, verified working
print("With AutoMod EdgeList Shape: ", edgeList.shape)
noAutoModEdgeList = edgeList[(edgeList != 'AutoModerator').all(1)]
noAutoModEdgeList.to_csv("Edgelists/No AutoMod/{}-edgeListNOAutoMod-CommentsToPosts.csv".format(subName), index=False)
print("No AutoMod EdgeList Shape: ", noAutoModEdgeList.shape)

In [ ]:
import scipy.stats as stats
###########################Some Statistics of the Data to help pruning#######################
#Using Cleaned comments therefore only comments that are not made by the same person as the post
commentStats = commentsOfPostsDF.copy()
#Convert unix time to datetime.
commentStats['created_utc'] = pandas.to_datetime(commentStats['created_utc'],unit='s')

#Using Cleaned Posts
postsStats = postsDataDF.copy()
postsStats['created_utc'] = pandas.to_datetime(postsStats['created_utc'],unit='s')


#####Posts per day over the 3 month period#####
numPostsDay = postsStats.groupby(postsStats['created_utc'].dt.date).size().reset_index(name='Count')
##################################################



#####Comments per day over the 3 month period#####
numCommsDay = commentStats.groupby(commentStats['created_utc'].dt.date).size().reset_index(name='Count')
##################################################



#####Bar Chart of number of comments per user#####
authorComments = commentStats['author'].value_counts()
#Convert back to dataframe
authorComments = authorComments.to_frame()
authorComments.reset_index(inplace=True)
authorComments.columns = ['author', 'count']
##################################################





#All Plots
numPostsDay.plot.bar(x='created_utc', figsize=(15,7), title='PostsPerDay')
numCommsDay.plot.bar(x='created_utc', figsize=(15,7), title='CommentsPerDay')
authorComments[:100].plot.bar(x='author', y='count',figsize=(15,7), title='Author comment count')